In [1]:
OPENSLIDE_PATH = r'C:/Users/Arist/Downloads/openslide-bin-4.0.0.3-windows-x64/openslide-bin-4.0.0.3-windows-x64/bin'

import os
if hasattr(os, 'add_dll_directory'):
    # Windows
    with os.add_dll_directory(OPENSLIDE_PATH):
        print('yes')
        import openslide
else:
    import openslide

import xml.etree.ElementTree as ET
import numpy as np
from PIL import Image, ImageDraw
import math
import cv2

yes


In [10]:
def create_binary_mask(annotation_file, dimensions):
    # Load WSI image
    mask_width, mask_height = dimensions

    # Create a blank mask
    mask = np.zeros((mask_height, mask_width), dtype=np.uint8)

    # Parse annotation XML file
    tree = ET.parse(annotation_file)
    root = tree.getroot()

    # Loop through annotations
    for annotation in root.findall('.//Annotation'):
        coords = []
        for coord in annotation.findall('.//Coordinate'):
            x = float(coord.attrib['X'])
            y = float(coord.attrib['Y'])
            # Convert coordinates to image space
            x_img = int(x)
            y_img = int(y)
            coords.append((x_img, y_img))
        # Convert coords list to array
        pts = np.array(coords, np.int32)
        pts = pts.reshape((-1, 1, 2))
        # Draw filled polygon
        cv2.fillPoly(mask, [pts], 255)

    return mask

In [11]:
# Example usage
annotation_file = 'test_files/RECHERCHE-003.xml'
wsi_file = 'test_files/RECHERCHE-003.svs'
#binary_mask = create_binary_mask(annotation_file, wsi_file)

In [12]:
annotation_file = 'test_files/RECHERCHE-003.xml'
slide = openslide.OpenSlide(wsi_file)
dim = slide.dimensions
mask = create_binary_mask(annotation_file, dim)

In [13]:
im = Image.fromarray(mask, 'L')
im = im.resize((int(dim[0]/2.0), int(dim[1]/2.0)), Image.Resampling.NEAREST)
im.show()

In [15]:
num_ones = np.count_nonzero(mask)
num_ones

39612937

In [11]:
s = mask.shape[0]*mask.shape[1]
num_zero = s - num_ones
num_zero

6856053565